In [ ]:
#importing libraries
import pandas as pd
import re
from collections import defaultdict, Counter
from itertools import islice
import os

In [ ]:
import pandas as pd
import re
import os
from IPython.display import FileLink

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/imdb reviews/IMDB Dataset.csv')  # Path to your CSV file

# Preprocess the text data
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize (split by whitespace)
    tokens = text.split()
    return tokens

# Apply preprocessing to the 'review' column
df['processed_reviews'] = df['review'].apply(preprocess_text)

# Keep the original review, processed reviews, and sentiment columns
processed_df = df[['review', 'processed_reviews', 'sentiment']]

# Save the processed data to a new CSV file
processed_df.to_csv('processed_reviews_with_sentiment.csv', index=False)

# Provide a downloadable link to the processed CSV file
download_path = os.path.join(os.getcwd(), 'processed_reviews_with_sentiment.csv')
print(f"Processed data saved to: {download_path}")

# Display download link in a notebook environment
FileLink('processed_reviews_with_sentiment.csv')


Processed data saved to: /content/processed_reviews_with_sentiment.csv


/content/processed_reviews_with_sentiment.csv

In [ ]:

import pandas as pd


df = pd.read_csv('/content/processed_reviews_with_sentiment.csv')

# Remove the 'review' column
df = df.drop('review', axis=1)

# Save the modified DataFrame back to a CSV file (optional)
df.to_csv('processed_reviews_without_review_column.csv', index=False)


print(df.head())


                                   processed_reviews sentiment
0  ['one', 'of', 'the', 'other', 'reviewers', 'ha...  positive
1  ['a', 'wonderful', 'little', 'production', 'br...  positive
2  ['i', 'thought', 'this', 'was', 'a', 'wonderfu...  positive
3  ['basically', 'theres', 'a', 'family', 'where'...  negative
4  ['petter', 'matteis', 'love', 'in', 'the', 'ti...  positive


In [ ]:
# Load the preprocessed CSV file
df = pd.read_csv('/content/processed_reviews_without_review_column.csv')

# Display the first few rows to check the content
print(df.head())

                                   processed_reviews sentiment
0  ['one', 'of', 'the', 'other', 'reviewers', 'ha...  positive
1  ['a', 'wonderful', 'little', 'production', 'br...  positive
2  ['i', 'thought', 'this', 'was', 'a', 'wonderfu...  positive
3  ['basically', 'theres', 'a', 'family', 'where'...  negative
4  ['petter', 'matteis', 'love', 'in', 'the', 'ti...  positive


In [ ]:
import re
from collections import defaultdict

# Function to create n-grams
def create_ngrams(tokens, n):
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [' '.join(ngram) for ngram in ngrams]

# Function to build the n-gram model
def build_ngram_model(reviews, n=1):
    ngram_counts = defaultdict(int)

    for tokens in reviews:
        ngrams = create_ngrams(tokens, n)
        for ngram in ngrams:
            ngram_counts[ngram] += 1

    return ngram_counts

# Convert the processed reviews from string back to list
df['processed_reviews'] = df['processed_reviews'].apply(eval)

# Build the unigram, bigram, and trigram models
unigram_model = build_ngram_model(df['processed_reviews'], 1)
bigram_model = build_ngram_model(df['processed_reviews'], 2)
trigram_model = build_ngram_model(df['processed_reviews'], 3)

# Check a sample of n-gram counts
print("Sample bigram counts:", list(bigram_model.items())[:10])


Sample bigram counts: [('one of', 16152), ('of the', 77006), ('the other', 4776), ('other reviewers', 170), ('reviewers has', 2), ('has mentioned', 13), ('mentioned that', 73), ('that after', 146), ('after watching', 610), ('watching just', 34)]


In [ ]:
# Function to predict the next word using the unigram, bigram, or trigram model
def predict_next_word(ngram_model, previous_words=None, n=1):
    if n == 1:
        # For the unigram model, predict the most frequent word in the corpus
        most_frequent_word = max(ngram_model, key=ngram_model.get)
        return most_frequent_word
    elif n == 2:
        # For the bigram model, predict the next word based on the previous word
        query = previous_words
    elif n == 3:
        # For the trigram model, predict the next word based on the previous two words
        query = ' '.join(previous_words.split()[-2:])

    # Find all n-grams that start with the given previous words
    ngrams = [ngram for ngram in ngram_model if ngram.startswith(query)]

    if ngrams:
        # Return the most frequent next word based on n-gram frequency
        next_word = max(ngrams, key=lambda ngram: ngram_model[ngram]).split()[-1]
        return next_word
    else:
        return None


In [ ]:
# Unigram Example: Predict the most frequent word
most_frequent_word = predict_next_word(unigram_model, n=1)
print(f'The most frequent word in the corpus (unigram model): {most_frequent_word}')

# Bigram Example: Predict the next word using the bigram model
previous_word = 'movie'
next_word_bigram = predict_next_word(bigram_model, previous_word, 2)
print(f'Next word prediction using bigram model: {next_word_bigram}')

# Trigram Example: Predict the next word using the trigram model
previous_words = 'the movie is'
next_word_trigram = predict_next_word(trigram_model, previous_words, 3)
print(f'Next word prediction using trigram model: {next_word_trigram}')


The most frequent word in the corpus (unigram model): the
Next word prediction using bigram model: is
Next word prediction using trigram model: a


In [ ]:
# Separate positive and negative reviews
positive_reviews = df[df['sentiment'] == 'positive']['processed_reviews']
negative_reviews = df[df['sentiment'] == 'negative']['processed_reviews']


In [ ]:
# Build n-gram models for positive and negative reviews
unigram_model_pos = build_ngram_model(positive_reviews, 1)
unigram_model_neg = build_ngram_model(negative_reviews, 1)

bigram_model_pos = build_ngram_model(positive_reviews, 2)
bigram_model_neg = build_ngram_model(negative_reviews, 2)

trigram_model_pos = build_ngram_model(positive_reviews, 3)
trigram_model_neg = build_ngram_model(negative_reviews, 3)


In [ ]:
import math

# Updated function with Laplace Smoothing
def classify_review_naive_bayes(review_tokens, ngram_model_pos, ngram_model_neg, n=1, pos_total_count=None, neg_total_count=None, alpha=1):
    # Initialize probabilities with the priors (log)
    pos_prob = math.log(len(positive_reviews) / len(df))  # Prior for positive class
    neg_prob = math.log(len(negative_reviews) / len(df))  # Prior for negative class

    # Loop over n-grams in the review
    for i in range(len(review_tokens) - n + 1):
        ngram = ' '.join(review_tokens[i:i+n])

        # Use Laplace smoothing (add alpha)
        pos_count = ngram_model_pos.get(ngram, 0) + alpha  # Laplace smoothing for positive n-grams
        neg_count = ngram_model_neg.get(ngram, 0) + alpha  # Laplace smoothing for negative n-grams

        # Update the log-probabilities with smoothing
        pos_prob += math.log(pos_count / (pos_total_count + alpha * len(ngram_model_pos)))
        neg_prob += math.log(neg_count / (neg_total_count + alpha * len(ngram_model_neg)))

    # Return the class with the higher probability
    return 'positive' if pos_prob > neg_prob else 'negative'


In [ ]:
# Precompute the total count of n-grams in the positive and negative models
pos_total_count_unigram = sum(unigram_model_pos.values())
neg_total_count_unigram = sum(unigram_model_neg.values())

pos_total_count_bigram = sum(bigram_model_pos.values())
neg_total_count_bigram = sum(bigram_model_neg.values())

pos_total_count_trigram = sum(trigram_model_pos.values())
neg_total_count_trigram = sum(trigram_model_neg.values())


In [ ]:
# Function to classify reviews using precomputed n-gram total counts
def classify_reviews(df, ngram_model_pos, ngram_model_neg, n=1, pos_total_count=None, neg_total_count=None):
    predictions = []

    for review_tokens in df['processed_reviews']:
        pred_label = classify_review_naive_bayes(review_tokens, ngram_model_pos, ngram_model_neg, n, pos_total_count, neg_total_count)
        predictions.append(pred_label)

    return predictions


In [ ]:
# Example: Classify using the optimized bigram model (n=2)
df['predicted_sentiment'] = classify_reviews(
    df,
    bigram_model_pos,
    bigram_model_neg,
    n=2,
    pos_total_count=pos_total_count_bigram,
    neg_total_count=neg_total_count_bigram
)


In [ ]:
# Generic function to classify reviews and suggest corrections
def suggest_correction_generic(df, ngram_model_pos, ngram_model_neg, pos_total_count, neg_total_count, n=1):
    corrections = []

    # Classify each review and compare with actual sentiment
    for idx, row in df.iterrows():
        review_tokens = row['processed_reviews']
        true_label = row['sentiment']  # Actual label from the dataset

        # Classify the review using the n-gram model (unigram, bigram, trigram, etc.)
        predicted_label = classify_review_naive_bayes(
            review_tokens,
            ngram_model_pos,
            ngram_model_neg,
            n=n,
            pos_total_count=pos_total_count,
            neg_total_count=neg_total_count
        )

        # Check if the predicted label is incorrect
        if predicted_label != true_label:
            corrections.append({
                'review': ' '.join(review_tokens),
                'predicted_sentiment': predicted_label,
                'true_sentiment': true_label,
                'suggested_correction': f"Correct to {true_label}"
            })

    return corrections


In [ ]:
# Function to build n-gram models for positive and negative reviews
def build_ngram_models(positive_reviews, negative_reviews, n):
    # Build n-gram models for positive and negative reviews
    ngram_model_pos = build_ngram_model(positive_reviews, n)
    ngram_model_neg = build_ngram_model(negative_reviews, n)

    # Precompute total n-gram counts
    pos_total_count = sum(ngram_model_pos.values())
    neg_total_count = sum(ngram_model_neg.values())
    # Add these lines after building n-gram models in run_ngram_classification
    print("Sample of Positive N-Gram Model (First 10 entries):")
    print(list(ngram_model_pos.items())[:10])

    print("Sample of Negative N-Gram Model (First 10 entries):")
    print(list(ngram_model_neg.items())[:10])


    return ngram_model_pos, ngram_model_neg, pos_total_count, neg_total_count


In [ ]:
def run_ngram_classification(df, positive_reviews, negative_reviews, n):
    # Build the n-gram models and precompute total counts
    ngram_model_pos, ngram_model_neg, pos_total_count, neg_total_count = build_ngram_models(positive_reviews, negative_reviews, n)



    # Counters for correct and incorrect sentiment predictions
    correct_sentiment_prediction = 0
    wrong_sentiment_prediction = 0

    # Classify reviews and suggest corrections
    corrections = suggest_correction_generic(
        df,
        ngram_model_pos,
        ngram_model_neg,
        pos_total_count,
        neg_total_count,
        n=n
    )

    # Check predictions and update counters
    for correction in corrections:
        predicted_sentiment = correction['predicted_sentiment']
        true_sentiment = correction['true_sentiment']

        # Compare predicted sentiment with true sentiment
        if predicted_sentiment == true_sentiment:
            correct_sentiment_prediction += 1  # Correct prediction
        else:
            wrong_sentiment_prediction += 1  # Incorrect prediction
            print(f"Review: {correction['review']}")
            print(f"Predicted: {predicted_sentiment}")

            print("---")




In [ ]:
#for running the model with different Ngram
run_ngram_classification(df, positive_reviews, negative_reviews, n=1)


Streaming output truncated to the last 5000 lines.
Predicted: negative
---
Review: tripping over i must say at first i was a little disappointed in the first few episodes but having faith in the show and abe forsythes unquestionable talent i continued to watch i can safely say im now glad that i did the story did develop quite well and all the characters have a strong base and most dont have any information missingbr br the only thing i can fault in this production is the somewhat annoying voice and pronunciation possessed by the character lizziebr br some good acting coupled with a stellar plot really gets this show over the line heres to hoping for another season
Predicted: negative
---
Review: humphrey bogart clearly did not want to be in this film and be forced to play a partmexican or he would have been suspended believe me he made the wrong choice presumably after the success of dodge city warners tried a followup with errol flynn and his usual list of buddies like alan hale guin